Here you upload the model architecture and weights created during model training. The model is used for predictions based on the test sample. The results will be saved to the `test_predictions.csv` file.

In [21]:
#!g1.1
import torch
import torchvision
import torchvision.transforms as transforms
from my_nn_model import Net
import pandas as pd

In [2]:
#!g1.1
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [3]:
#!g1.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
#!g1.1
net = Net()
net.to(device)
net.load_state_dict(torch.load('./cifar_net.pth'))

predictions = []
predicted_labels = []
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.append(predicted.tolist())
        predicted_labels.append([classes[predicted[j]] for j in range(batch_size)])

In [22]:
#!g1.1
final_pred = pd.DataFrame({'class_idx': [item for sublist in predictions for item in sublist],
                           'class': [item for sublist in predicted_labels for item in sublist]})

In [26]:
#!g1.1
final_pred.to_csv('test_predictions.csv')